In [0]:
# Load Libraries
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import pandas as pd 
import multiprocessing
import random

from sklearn.preprocessing import scale
from sklearn.preprocessing import OneHotEncoder
import timeit

import csv
import requests
from bs4 import BeautifulSoup

In [0]:
pip install requests

In [0]:
pip install beautifulsoup4

In [0]:
from datetime import datetime, timedelta
from threading import Thread
from time import sleep

class Temporizador(Thread):
    def __init__(self, hora, delay, funcion):
        super(Temporizador, self).__init__()
        self._estado = True
        self.hora = hora
        self.delay = delay
        self.funcion = funcion

    def stop(self):
        self._estado = False

    def run(self):
        aux = datetime.strptime(self.hora, '%H:%M:%S')
        hora = datetime.now()
        hora = hora.replace(hour = aux.hour, minute=aux.minute, second=aux.second, microsecond = 0)
        if hora <= datetime.now():
            hora += timedelta(days=1)

        while self._estado:
            if hora <= datetime.now():
                self.funcion()
                hora += timedelta(days=1)
            sleep(self.delay)

In [0]:
def scrape_data(url):

    response = requests.get(url, timeout=10)
    soup = BeautifulSoup(response.content, 'html.parser')

    table = soup.find_all('table')[0]

    rows = table.select('tbody > tr')

    header = [th.text.rstrip() for th in table.find_all('th')]

    with open('output6.csv', 'w') as csv_file:
        writer = csv.writer(csv_file)
        writer.writerow(header)
        for row in rows[0:]:
            data = [th.text.rstrip() for th in row.find_all('td')]
            writer.writerow(data)

if __name__=="__main__":
    url = "https://github.com/datadista/datasets/blob/master/COVID%2019/ccaa_covid19_datos_isciii.csv"
    scrape_data(url)


In [0]:
url = "https://github.com/datadista/datasets/blob/master/COVID%2019/ccaa_covid19_datos_isciii.csv"
scrape_data(url)

In [0]:
risk_by_region=pd.read_csv('/content/output6.csv')
risk_by_region

,Fecha,cod_ine,CCAA,Casos,PCR+,TestAc+,Hospitalizados,UCI,Fallecidos,Recuperados
NaN,2020-02-20,1,Andalucía,NaN,NaN,NaN,NaN,NaN,NaN,NaN
NaN,2020-02-20,2,Aragón,NaN,NaN,NaN,NaN,NaN,NaN,NaN
NaN,2020-02-20,3,Asturias,NaN,NaN,NaN,NaN,NaN,NaN,NaN
NaN,2020-02-20,4,Baleares,1.0,NaN,NaN,NaN,NaN,NaN,NaN
NaN,2020-02-20,5,Canarias,1.0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
NaN,2020-05-06,19,Melilla,NaN,119.0,13.0,44.0,3.0,2.0,108.0
NaN,2020-05-06,14,Murcia,NaN,1501.0,691.0,652.0,111.0,137.0,1527.0
NaN,2020-05-06,15,Navarra,NaN,4983.0,2027.0,2015.0,135.0,480.0,2732.0
NaN,2020-05-06,16,País Vasco,NaN,13041.0,3918.0,6797.0,561.0,1383.0,13070.0


In [0]:
current_risk_by_region.describe()

In [0]:
current_risk_by_region=current_risk_by_region.tail(19)
current_risk_by_region

,Fecha,cod_ine,CCAA,Casos,PCR+,TestAc+,Hospitalizados,UCI,Fallecidos,Recuperados
NaN,2020-05-06,1,Andalucía,NaN,12268.0,2602.0,6027.0,744.0,1294.0,8076.0
NaN,2020-05-06,2,Aragón,NaN,5258.0,1258.0,2564.0,262.0,800.0,2880.0
NaN,2020-05-06,3,Asturias,NaN,2326.0,773.0,2117.0,143.0,292.0,971.0
NaN,2020-05-06,4,Baleares,NaN,1929.0,94.0,1132.0,169.0,201.0,1345.0
NaN,2020-05-06,5,Canarias,NaN,2235.0,0.0,921.0,176.0,144.0,1258.0
NaN,2020-05-06,6,Cantabria,NaN,2220.0,422.0,1020.0,79.0,200.0,1758.0
NaN,2020-05-06,8,Castilla La Mancha,NaN,16184.0,6383.0,8796.0,613.0,2677.0,5895.0
NaN,2020-05-06,7,Castilla y León,NaN,17625.0,4912.0,8300.0,541.0,1864.0,7149.0
NaN,2020-05-06,9,Cataluña,NaN,51190.0,1315.0,28248.0,2891.0,5394.0,23231.0
NaN,2020-05-06,18,Ceuta,NaN,109.0,56.0,10.0,4.0,4.0,150.0


In [0]:
def Active(a,b,c,d): 
    x1=a+d-b-c
    return x1

def Inmnune(a,b): 
    x2=a+b
    return x2

def Actives_by_region(): 
      
    current_risk_by_region['Actives_by_region'] = current_risk_by_region.apply(lambda row : Active(row['PCR+'],row['Fallecidos'],row['Recuperados'],row['TestAc+']), axis = 1) 
   
    return current_risk_by_region

def Inmunes_by_region(): 
      
    current_risk_by_region['Inmunes_by_region'] = current_risk_by_region.apply(lambda row : Inmnune(row['TestAc+'],row['Recuperados']), axis = 1) 
   
    return current_risk_by_region


To calculate the infection rate of one area with regard to another, we apply the function Ro: Current Assets/Total Positive.

In [0]:
def Ro_(a,b): 
    Ro_=(a/b)
    return Ro_

def Ro_by_region(): 
      
    current_risk_by_region['Ro_by_region'] = current_risk_by_region.apply(lambda row : Ro_(row['Actives_by_region'],row['PCR+']), axis = 1) 
   
    return current_risk_by_region

In [0]:
current_risk_by_region['Actives_by_region']=current_risk_by_region.iloc[:,0:2].sum(axis=1)

In [0]:
def Zone(a): 
    b=current_risk_by_region['Actives_by_region'].sum()
    Zone=(a/b)*100
    return Zone

def Risk_by_region(): 
      
    current_risk_by_region['Risk_by_region'] = current_risk_by_region.apply(lambda row : Zone(row['Actives_by_region']), axis = 1) 
   
    return current_risk_by_region

In [0]:
def Prob_risk(a): 
    b=current_risk_by_region['Risk_by_region'].max()
    Prob_risk=(a/b)*100
    return Prob_risk

def Risk_by_region_final(): 
      
    current_risk_by_region['Risk_by_region_final'] = current_risk_by_region.apply(lambda row : Prob_risk(row['Risk_by_region']), axis = 1) 
   
    return current_risk_by_region

We apply the three functions that have been defined to the DataFrame.

In [0]:
current_risk_by_region=Actives_by_region()
current_risk_by_region=Inmunes_by_region()
current_risk_by_region=Ro_by_region()
current_risk_by_region=Risk_by_region()
current_risk_by_region=Risk_by_region_final()

In [0]:
c=current_risk_by_region['Actives_by_region'].sum()

102274.0

In [0]:
current_risk_by_region

,Fecha,cod_ine,CCAA,Casos,PCR+,TestAc+,Hospitalizados,UCI,Fallecidos,Recuperados,Actives_by_region,Inmunes_by_region,Ro_by_region,Risk_by_region,Risk_by_region_final
NaN,2020-05-06,1,Andalucía,NaN,12268.0,2602.0,6027.0,744.0,1294.0,8076.0,5500.0,10678.0,0.448321,5.377711,23.031826
NaN,2020-05-06,2,Aragón,NaN,5258.0,1258.0,2564.0,262.0,800.0,2880.0,2836.0,4138.0,0.539369,2.772943,11.876047
NaN,2020-05-06,3,Asturias,NaN,2326.0,773.0,2117.0,143.0,292.0,971.0,1836.0,1744.0,0.789338,1.795178,7.688442
NaN,2020-05-06,4,Baleares,NaN,1929.0,94.0,1132.0,169.0,201.0,1345.0,477.0,1439.0,0.247278,0.466394,1.997487
NaN,2020-05-06,5,Canarias,NaN,2235.0,0.0,921.0,176.0,144.0,1258.0,833.0,1258.0,0.372707,0.814479,3.488275
NaN,2020-05-06,6,Cantabria,NaN,2220.0,422.0,1020.0,79.0,200.0,1758.0,684.0,2180.0,0.308108,0.668792,2.864322
NaN,2020-05-06,8,Castilla La Mancha,NaN,16184.0,6383.0,8796.0,613.0,2677.0,5895.0,13995.0,12278.0,0.864743,13.683830,58.605528
NaN,2020-05-06,7,Castilla y León,NaN,17625.0,4912.0,8300.0,541.0,1864.0,7149.0,13524.0,12061.0,0.767319,13.223302,56.633166
NaN,2020-05-06,9,Cataluña,NaN,51190.0,1315.0,28248.0,2891.0,5394.0,23231.0,23880.0,24546.0,0.466497,23.349043,100.000000
NaN,2020-05-06,18,Ceuta,NaN,109.0,56.0,10.0,4.0,4.0,150.0,11.0,206.0,0.100917,0.010755,0.046064


In [0]:
df=pd.DataFrame()

In [0]:
df=current_risk_by_region[['CCAA','Risk_by_region_final']]

In [0]:
df=df.reset_index()
df=df.drop(columns='index')
df

,CCAA,Risk_by_region_final
0,Andalucía,23.031826
1,Aragón,11.876047
2,Asturias,7.688442
3,Baleares,1.997487
4,Canarias,3.488275
5,Cantabria,2.864322
6,Castilla La Mancha,58.605528
7,Castilla y León,56.633166
8,Cataluña,100.000000
9,Ceuta,0.046064


In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
df.to_csv('Risk_by_region.csv',index=False)
!cp Risk_by_region.csv "/content/drive/My Drive"